In [70]:
# def parse_m2(m2_path):
#     """
#     Parse an M2 file into a list of dicts, each with:
#       - 'src':   the original (erroneous) sentence
#       - 'edits': list of edits, each dict:
#           - 'start': int character offset
#           - 'end':   int character offset
#           - 'etype': str error type
#           - 'corr':  str replacement text
#     """
#     examples = []
#     with open(m2_path, encoding='utf-8') as f:
#         src = None
#         edits = []
#         for line in f:
#             line = line.rstrip("\n")
#             # blank line → end of this example
#             if not line:
#                 if src is not None:
#                     examples.append({"input": src, "edits": edits})
#                 src, edits = None, []
#                 continue

#             if line.startswith("S "):
#                 # Source sentence
#                 src = line[2:]
#             elif line.startswith("A "):
#                 # Annotation line → strip off the "A "
#                 ann = line[2:]
#                 parts = ann.split("|||")
#                 # parts[0] is "start end"
#                 offsets = parts[0].split()
#                 if len(offsets) != 2:
#                     # malformed, skip
#                     continue
#                 start, end = int(offsets[0]), int(offsets[1])

#                 etype = parts[1]
#                 corr  = parts[2]

#                 # Skip noop edits (no change)
#                 if etype.lower() == "noop" or (start == -1 and end == -1):
#                     continue

#                 edits.append({
#                     "start": start,
#                     "end":   end,
#                     "etype": etype,
#                     "corr":  corr
#                 })
#         # handle case where file doesn't end with a blank line
#         if src is not None:
#             examples.append({"input": src, "edits": edits})

#     return examples


# def apply_edits_to_sentence(src, edits):
#     """
#     Apply token-level edits to a tokenized source string.

#     Args:
#       src   (str): whitespace-tokenized sentence
#       edits (list of dict): each with 'start','end','corr'

#     Returns:
#       str: the corrected sentence (still whitespace-tokenized)
#     """
#     # 1) split into tokens & build char-span map
#     tokens = src.split()
#     spans = []
#     cursor = 0
#     for tok in tokens:
#         # find this token at or after cursor
#         pos = src.find(tok, cursor)
#         if pos < 0:
#             raise ValueError(f"Token {tok!r} not found in src starting at {cursor}")
#         spans.append((pos, pos + len(tok)))
#         cursor = pos + len(tok)

#     corrected = src
#     # 2) apply edits in descending token-index order
#     for e in sorted(edits, key=lambda x: x["start"], reverse=True):
#         st, en, corr = e["start"], e["end"], e["corr"]
#         # compute char positions
#         if st < len(spans):
#             char_start = spans[st][0]
#         else:
#             # insertion at end of sentence
#             char_start = len(corrected)

#         if en == 0:
#             # insertion before 1st token
#             char_end = 0
#         elif en <= len(spans):
#             # normal span; end is exclusive token index → end at end-of-token(en-1)
#             char_end = spans[en - 1][1]
#         else:
#             # deletion spanning past last token
#             char_end = len(corrected)

#         # splice it in
#         corrected = corrected[:char_start] + corr + corrected[char_end:]

#     return corrected

### Train Dataset

In [71]:
# # Example usage:
# train_m2_file = "ABC.train.gold.bea19.m2"
# examples = parse_m2(train_m2_file)

# examples[0]

In [72]:
# # build a list of corrected sentences
# corrected_sentences = []
# for i,ex in enumerate(examples):
#     examples[i]['id']=i+1
#     src  = ex["input"]
#     corr = apply_edits_to_sentence(src, ex["edits"])
#     examples[i]['output']=corr
#     corrected_sentences.append(corr)

# # print them out
# for i,o in enumerate(corrected_sentences):
#     print(examples[i]['input'])
#     for j,edit in enumerate(examples[i]['edits']):
#         print(edit)
#     print(o)
#     print()
#     if i>5:
#         break


In [73]:
# from datasets import Dataset

In [74]:
# train_dataset= Dataset.from_list(examples)
# train_dataset

### Validation Dataset

In [75]:
# val_m2_file='ABCN.dev.gold.bea19.m2'
# examples= parse_m2(val_m2_file)
# len(examples)

In [76]:
# # build a list of corrected sentences
# corrected_sentences = []
# for i,ex in enumerate(examples):
#     examples[i]['id']=i+1
#     src  = ex["input"]
#     corr = apply_edits_to_sentence(src, ex["edits"])
#     examples[i]['output']=corr
#     corrected_sentences.append(corr)

# # print them out
# for i,o in enumerate(corrected_sentences):
#     print(examples[i]['input'])
#     for j,edit in enumerate(examples[i]['edits']):
#         print(edit)
#     print(o)
#     print()
#     if i>5:
#         break


In [77]:
# val_dataset= Dataset.from_list(examples)
# val_dataset

#### USED CODE FROM:

https://www.cl.cam.ac.uk/research/nl/bea2019st/#data

In [1]:
import argparse

# Apply the edits of a single annotator to generate the corrected sentences.
def get_train_val(args):
    m2 = open(args['m2_file']).read().strip().split("\n\n")
    # out = open(args.out, "w")
    # Do not apply edits with these error types
    skip = {"noop", "UNK", "Um"}
    inputs=[]
    output=[]
    edits=[]
    for sent in m2:
        sent = sent.split("\n")
        
        cor_sent = sent[0].split()[1:] # Ignore "S "
        # inputs.append(' '.join(cor_sent))
        inputs.append(sent[0][2:])
        e = sent[1:]
        edits.append(e)
        offset = 0
        for edit in e:
            edit = edit.split("|||")
            if edit[1] in skip: continue # Ignore certain edits
            coder = int(edit[-1])
            if coder != args['id']: 
                print('different coder')
                continue # Ignore other coders
            span = edit[0].split()[1:] # Ignore "A "
            start = int(span[0])
            end = int(span[1])
            cor = edit[2].split()
            cor_sent[start+offset:end+offset] = cor
            offset = offset-(end-start)+len(cor)
        output.append([" ".join(cor_sent)])
        # out.write(" ".join(cor_sent)+"\n")
    return inputs,edits,output

### Train Dataset

In [2]:
from datasets import Dataset, load_from_disk

In [3]:
args={
    'm2_file':'ABC.train.gold.bea19.m2',
    'id':0
}
inputs,edits,output=get_train_val(args)


In [4]:
examples={
    'id':[str(i) for i in range(len(inputs))],
    'input':inputs,
    'edits':edits,
    'output':output
}

In [5]:
train=Dataset.from_dict(examples)
train

Dataset({
    features: ['id', 'input', 'edits', 'output'],
    num_rows: 34308
})

In [6]:
train[0]

{'id': '0',
 'input': 'My town is a medium size city with eighty thousand inhabitants .',
 'edits': ['A 5 6|||R:OTHER|||- sized|||REQUIRED|||-NONE-|||0'],
 'output': ['My town is a medium - sized city with eighty thousand inhabitants .']}

In [7]:
from tqdm import tqdm

filtered = train.filter(lambda x: bool(x['output']))
print(len(filtered))

Filter:   0%|          | 0/34308 [00:00<?, ? examples/s]

34308


In [8]:
train

Dataset({
    features: ['id', 'input', 'edits', 'output'],
    num_rows: 34308
})

### Test Dataset

In [9]:
import argparse

def get_test(m2_file):
    # Read and split on blank lines
    with open(m2_file, encoding="utf-8") as f:
        blocks = f.read().strip().split("\n\n")

    # Error types to ignore
    skip = {"noop", "UNK", "Um"}

    inputs = []
    edits = []
    output = []

    # First pass: parse inputs & edits for each block
    for block in blocks:
        lines = block.split("\n")
        # original sentence (drop leading "S ")
        orig = lines[0][2:]
        inputs.append(orig)

        # raw edit triples/quintuples
        this_edits = [line.split("|||") for line in lines[1:]]
        edits.append(this_edits)

    # For each sentence, build one list of corrected forms (one per coder)
    i=0
    for orig, this_edits in zip(inputs, edits):
        # print(f'i: {i}')
        # print(f'This edit: {this_edits}')
        tokens = orig.split()
        # find all coder IDs with at least one non-skipped edit here
        coder_ids = sorted({
            int(ed[-1])
            for ed in this_edits
        })
        per_sentence = []
        if not coder_ids:
            # print(f'No Coders found')
            per_sentence=[orig]
        # print(coder_ids)
        f=False
        
        for cid in coder_ids:
            # f=False
            cor_sent = tokens.copy()
            # print(f'cid: {cid}')
            offset = 0
            for span_str, err_type, corr_str, *rest, coder_str in this_edits:
                if err_type in skip or int(coder_str) != cid:
                    # print('Encountered')
                    # print(f)
                    # f=True
                    continue
                # parse "A start end"
                _, s, e = span_str.split()
                start, end = int(s), int(e)
                cor_toks = corr_str.split()

                # apply with offset
                cor_sent[start+offset : end+offset] = cor_toks
                offset += len(cor_toks) - (end - start)
            
            per_sentence.append(" ".join(cor_sent))
            # print(f'per_sentence: {per_sentence}')
            # if f:
            #     print('Breaking')
            #     break
        
        i+=1
        output.append(per_sentence)
        # print(f'Output: {output}')
        # if i>8:
        #     break
        # if f:
        #     print(per_sentence)
        #     break

    return inputs, edits, output



In [10]:
inputs, edits, output= get_test('test.m2')

In [11]:
examples={
    'id':[str(i) for i in range(len(inputs))],
    'input':inputs,
    'edits':edits,
    'output':output
}

In [12]:
test=Dataset.from_dict(examples)
test

Dataset({
    features: ['id', 'input', 'edits', 'output'],
    num_rows: 1312
})

In [13]:
filtered = test.filter(lambda x: bool(x['output']))
print(len(filtered))

Filter:   0%|          | 0/1312 [00:00<?, ? examples/s]

1312


In [14]:
test

Dataset({
    features: ['id', 'input', 'edits', 'output'],
    num_rows: 1312
})

### Validation Dataset

In [15]:
args={
    'm2_file':'ABCN.dev.gold.bea19.m2',
    'out':'corrected',
    'id':0
}
inputs,edits,output=get_train_val(args)


In [16]:
examples={
    'id':[str(i) for i in range(len(inputs))],
    'input':inputs,
    'edits':edits,
    'output':output
}

In [17]:
val=Dataset.from_dict(examples)
val

Dataset({
    features: ['id', 'input', 'edits', 'output'],
    num_rows: 4384
})

In [18]:
filtered = val.filter(lambda x: bool(x['output']))
print(len(filtered))

Filter:   0%|          | 0/4384 [00:00<?, ? examples/s]

4384


In [19]:
val

Dataset({
    features: ['id', 'input', 'edits', 'output'],
    num_rows: 4384
})

### Save the datasets

In [20]:
dir_path='/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/Small-LLM-Reasoning/datasets/gec/'

In [21]:
split='val'
val.save_to_disk(dir_path+split)

Saving the dataset (0/1 shards):   0%|          | 0/4384 [00:00<?, ? examples/s]

In [22]:
split='train'
train.save_to_disk(dir_path+split)

Saving the dataset (0/1 shards):   0%|          | 0/34308 [00:00<?, ? examples/s]

In [23]:
split='test'
test.save_to_disk(dir_path+split)

Saving the dataset (0/1 shards):   0%|          | 0/1312 [00:00<?, ? examples/s]

### Spliting train set into feedback and train set

In [48]:
train=load_from_disk('../datasets/gec/train/')
train

Dataset({
    features: ['id', 'input', 'edits', 'output'],
    num_rows: 34308
})

In [49]:
import json
task='gec'
task_prompt_path=f'../prompts/{task}.json'
with open(task_prompt_path) as fp:
    task_prompt = json.load(fp)
task_prompt

{'task_prompt': 'Correct grammatical errors in the text by first providing a response, followed by an explanation. Please use this template for the explanation: "The word X should be deleted/inserted/replaced by Y because ..."',
 'task_prompt1': 'Given an input text, the goal is to detect and correct grammatical errors in the text. First explain your reasoning by describing the grammatical errors and how to fix them and then, provide the corrected text.\n\nYour response should end with "The corrected text is: [answer]" where [answer] is the grammatically correct version of the input text.',
 'system_msg': '',
 'user_msg': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{question}\n\n### Response:\n',
 'assistant_msg': '{rationale} The corrected text is {response}\n',
 'few_shot': [{'id': '8778',
   'input': 'way to move from pl

In [50]:
few_shot_ids=[]
for i in task_prompt['few_shot']:
    few_shot_ids.append(i['id'])
few_shot_ids


['8778', '21174', '17866', '19539', '25488']

In [51]:
train=train.filter(lambda row:row['id'] not in few_shot_ids)
train

Dataset({
    features: ['id', 'input', 'edits', 'output'],
    num_rows: 34303
})

In [52]:
import random

feedback=train.shuffle(42).select(range(400))
feedback

Dataset({
    features: ['id', 'input', 'edits', 'output'],
    num_rows: 400
})

In [53]:
feedback.save_to_disk('../datasets/gec/feedback-400/')

Saving the dataset (0/1 shards):   0%|          | 0/400 [00:00<?, ? examples/s]

In [54]:
import random

feedback=train.shuffle(42).select(range(100))
feedback

Dataset({
    features: ['id', 'input', 'edits', 'output'],
    num_rows: 100
})

In [55]:
feedback.save_to_disk('../datasets/gec/feedback-100/')

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

In [56]:
feedback=train.shuffle(42).select(range(1600))
feedback

Dataset({
    features: ['id', 'input', 'edits', 'output'],
    num_rows: 1600
})

In [57]:
feedback.save_to_disk('../datasets/gec/feedback-1600/')

Saving the dataset (0/1 shards):   0%|          | 0/1600 [00:00<?, ? examples/s]

### Creating m2 files for feedback set

In [71]:
split='feedback-400'
data=load_from_disk(f'../datasets/gec/{split}/')
data

Dataset({
    features: ['id', 'input', 'edits', 'output'],
    num_rows: 400
})

In [72]:
data[2]

{'id': '33345',
 'input': 'In the first conic part it is possible to feed with the fuel ( i.e. the pellet ) that leads to the combustion chamber where the pellet will burn to generate the heat necessary to warm up the air .',
 'edits': ['A 3 4|||R:MORPH|||conical|||REQUIRED|||-NONE-|||0',
  'A 5 5|||M:PUNCT|||,|||REQUIRED|||-NONE-|||0',
  'A 10 11|||R:PREP|||in|||REQUIRED|||-NONE-|||0'],
 'output': ['In the first conical part , it is possible to feed in the fuel ( i.e. the pellet ) that leads to the combustion chamber where the pellet will burn to generate the heat necessary to warm up the air .']}

In [73]:
cnt=0
with open(f'../datasets/gec/{split}/{split}.m2', 'w', encoding='utf-8') as fout:
        for ex in data:
            cnt+=1
            # 1) write the source line
            #    M2 uses tokenized input; if your source is untokenized,
            #    you may need to tokenize (e.g. simple .split()).
            fout.write("S " + ex['input'].strip() + "\n")
            # 2) write each edit line
            for edit in ex['edits']:
                fout.write(edit.strip() + "\n")
            # 3) blank line between examples
            fout.write("\n")

In [74]:
cnt

400